<a href="https://colab.research.google.com/github/dzzzzdy/Big_Data_analytics/blob/main/Chat_Request_pysparketl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pwd

!ls

!ls /usr/lib/jvm/

/content
sample_data
java-1.11.0-openjdk-amd64  java-11-openjdk-amd64


In [8]:
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

!tar -xvzf spark-3.4.1-bin-hadoop3.tgz

spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

# New Section

In [9]:
!pip install -q findspark

!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=311d4a98737cb5a6d5f93935a27a7d5c1b0e79035f3738580ad77d176eb6d34d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [89]:
import findspark
findspark.init()

In [109]:
#import library
import pandas as pd
from pyspark.sql import SparkSession
import sys
import pyspark.sql.functions as F
from pyspark.sql.functions import col,json_tuple, to_json, from_json, when,  posexplode, arrays_zip, row_number, to_timestamp, udf, lit, from_unixtime, datediff, split, first, monotonically_increasing_id, round
from pyspark.sql import Window
from pyspark.sql import SparkSession, types
from pyspark.sql.types import StringType

In [124]:
history_chat_path = "/content/history_chat_cleaned.csv"

# create a Spark session
spark = SparkSession.builder.appName('ChatRequest').getOrCreate()

# load the CSV data into a DataFrame
history_chat = spark.read.csv(history_chat_path, header = True, inferSchema = True)

history_chat.show(5)

+---------+------------------+------------------+--------------------+-----------------+-------------+--------------------+--------------------+------------------+--------------------+-------------+--------------------+-------------+-------------+--------+--------------------+---------------+----------+--------------------+------------+----------+----------------+------------+--------------+--------------+-------------+--------------------+--------------------+-----------+---------------+---------+--------------------+---+----------+-------+----------------+---------+-----+-----------+-------------+----------------+---------------+-------+--------------------+----------------------+----------+------------+---------+--------------------+-----------------+-------------------+----------------+--------------------+-----+-------+--------------------+---------------------+-------+-----------+----------------+--------+---------------+---------------+-------+--------------+----------------+---

In [125]:
history_transcript_path = '/content/history_transcript_cleaned.csv'
history_transcript = spark.read.csv(history_transcript_path, header = True, inferSchema = True)
history_transcript.show()

+------------------+------------------+-------+--------------------+-------------+-------+----------+------+-----------+--------+-----------+----------------+---------+--------------------+
|      engagementId|engagementSequence|lineSeq|                time|        timeL| source|   agentId|   mcs|mcsRawScore|textType|controlType|cannedAnswerType|       by|                text|
+------------------+------------------+-------+--------------------+-------------+-------+----------+------+-----------+--------+-----------+----------------+---------+--------------------+
|153572514298919339|                 0|     20|2023-08-01 01:39:...|1690853975671| system|         0|  null|       null|   plain|          4|               0|     info|Thank you for cho...|
|153572514298919339|                 0|     26|2023-08-01 01:39:...|1690853976150| system|         0|  null|       null|   plain|          3|               0|     info|You are now chatt...|
|153572514298919339|                 0|     32|202

In [126]:
combined_table = history_chat.join(history_transcript, on = ['engagementId', 'engagementSequence'])

windowNoOrdered = Window.partitionBy("EngagementID", "EngagementSequence")
windowOrdered = Window.partitionBy("EngagementID", "EngagementSequence").orderBy("LineSeq")

history_chat = combined_table.withColumn("is_freehand", F.when((col('source') == 'agent') & (~col('text').like('<span>%')), 1).otherwise(0)) \
                            .withColumn("is_agent_script", F.when((col('source') == 'agent') & (col('text').like('<span>%')), 1).otherwise(0)) \
                            .withColumn("is_agent_after_visitor", F.when((col("source") == "agent") & (F.lag(combined_table["source"]).over(windowOrdered) == "visitor"), 1).otherwise(0)) \
                            .withColumn('is_agent', F.when((col("source") == "agent"), 1).otherwise(0)) \
                            .withColumn("num_agent_freehand", F.sum("is_freehand").over(windowNoOrdered)) \
                            .withColumn("num_agent_scripted", F.sum("is_agent_script").over(windowNoOrdered)) \
                            .withColumn("num_agent_responses", F.sum("is_agent_after_visitor").over(windowOrdered)) \
                            .withColumn('num_cust_reponses', F.sum("is_agent").over(windowOrdered)) \
                            .withColumn('min_timeL_for_agent', F.min(F.when(col('source') == 'agent', col('timeL'))).over(windowNoOrdered)) \
                            .withColumn('initial_agent_response_time', round((col('min_timeL_for_agent') - col('startTimeL'))/1000)) \
                            .withColumn("visitor_timeL_lag",F.when((F.lag("source").over(windowOrdered) == "visitor"), F.lag("timeL").over(windowOrdered))) \
                            .withColumn("time_difference",F.when(col('source') == 'agent', col('timeL') - col('visitor_timeL_lag'))) \
                            .withColumn("MAX_AGENT_RESPONSE_TIME", F.max("time_difference").over(windowNoOrdered)) \
                            .withColumn('TOTAL_RESPONSE_TIME', F.sum('time_difference').over(windowNoOrdered)) \
                            .withColumn('AVG_RESPONSE_TIME', F.sum('time_difference').over(windowNoOrdered)) \
                            .withColumn("is_agent_after_visitor",F.when((F.col("source") == "agent") & (F.lag(F.col("source")).over(windowOrdered) == "visitor"),1).otherwise(0)) \
                            .withColumn('NUM_CUST_TO_AGENT_RESPONSES', F.sum('is_agent_after_visitor').over(windowOrdered))

In [127]:
#datatype transformation
'''history_chat = history_chat.withColumn('engagementid', col('engagementid').cast('int')) \
                           .withColumn('startTimeL', from_unixtime(col('startTimeL')/1000).cast('timestamp'))\
                           .withColumn('chatRequestedTimeL', from_unixtime(col('chatRequestedTimeL')/1000).cast('timestamp'))\
                           .withColumn('endTimeL', from_unixtime(col('endTimeL') / 1000).cast('timestamp'))'''

"history_chat = history_chat.withColumn('engagementid', col('engagementid').cast('int'))                            .withColumn('startTimeL', from_unixtime(col('startTimeL')/1000).cast('timestamp'))                           .withColumn('chatRequestedTimeL', from_unixtime(col('chatRequestedTimeL')/1000).cast('timestamp'))                           .withColumn('endTimeL', from_unixtime(col('endTimeL') / 1000).cast('timestamp'))"

In [132]:
disposition = history_chat.select('engagementIdSeq','survey-surveyType', 'survey-timeL', 'survey-time', 'isAgentSurvey', 'endTimeL', 'startTimeL')
str_list = split(disposition['survey-surveyType'], '::')
time_list = split(disposition['survey-time'], '::')
timeL_list = split(disposition['survey-timeL'], '::')

window = Window.partitionBy('engagementIdSeq').orderBy('index')
disposition = disposition.select('*', posexplode(arrays_zip(str_list, timeL_list, time_list)).alias('index', 'value')) \
                         .filter(col('value')['0'].startswith('operator')) \
                         .withColumn('row_num', row_number().over(window)).filter(col('row_num') == 1) \
                         .withColumn('disposition_epoch', col('value')['1']) \
                         .withColumn('disposition_dtm', col('value')['2']) \
                         .withColumn('disposition_time',
                                     when(col('disposition_epoch') < col('endTimeL'), 0). \
                                     otherwise(round((col('disposition_epoch') - col('endtimeL'))/1000))) \
                         .withColumn('handle_time',
                                      when(col('disposition_epoch') < col('endTimeL'),
                                      (round((col('endTimeL') - col('startTimeL'))/1000))). \
                                      otherwise(round((col('disposition_epoch') - col('startTimeL'))/ 1000))) \
                         .select('engagementIdSeq', 'disposition_epoch', 'disposition_dtm', 'disposition_time', 'handle_time')
history_chat = history_chat.join(disposition, on = 'engagementIdSeq', how = 'left')

AnalysisException: ignored

In [129]:
transfer_window = window.partitionBy('engagementId').orderBy('engagementSequence')

history_chat = history_chat.withColumn('leg_num', col('engagementSequence') + 1) \
                            .withColumn(
                                      'business_unit',
                                      when(col('skillName') == 'Broadband Billing', 'MTS_WLN_Billing_Webchat_Care_Support').
                                      when(col('skillName') == 'Broadband Service', 'MTS_WLN_Billing_Webchat_Care_Support').
                                      when(col('skillName') == 'Tech Support', 'MTS_WLN_Technical_Alternate_Contact_Team_Tech_Support').
                                      when(col('skillName') == 'Broadband Sales - Sitel', 'MTS_WLN_BB_Communications_Sales_Support').
                                      when(col('skillName') == 'Wireless Sales', 'MTS_WLN_BB_Communications_Sales_Support').
                                      when(col('skillName') == 'Business Broadband', 'MTS_WLN_Business_Channel_Support_Care_Support').
                                      otherwise(col('skillName'))
                                      ) \
                            .withColumnRenamed('skillname', 'agent_group') \
                            .withColumnRenamed('campaignName', 'business_rule') \
                            .withColumnRenamed('ChatStartPage','launch_page_marker') \
                            .withColumnRenamed('personalInfo-language', 'cust_lang') \
                            .withColumnRenamed('state', 'cust_prov') \
                            .withColumnRenamed('personalInfo-company', 'cust_company') \
                            .withColumnRenamed('personalInfo-email', 'cust_email') \
                            .withColumnRenamed('personalInfo-phone', 'cust_phone') \
                            .withColumn('bssp_registration', lit(None))\
                            .withColumnRenamed('chatRequestedTimeL','request_epoch') \
                            .withColumnRenamed('chatRequestedTime', 'request_dtm') \
                            .withColumn('assigned', lit(1)) \
                            .withColumn('abandoned', lit(None)) \
                            .withColumn('abandon_epoch', lit(None)) \
                            .withColumn('abandon_dtm', lit(None)) \
                            .withColumn('abandon_time', lit(None)) \
                            .withColumnRenamed('startTimeL', 'assign_epoch') \
                            .withColumnRenamed('startTime', 'assign_dtm') \
                            .withColumn('assign_time', (round((col('assign_epoch') - col('request_epoch'))/1000))) \
                            .withColumn(
                                      'assign_entertype',
                                      when(col('leg_num') == 1, 'Chat').
                                      otherwise('transfer')
                                      ) \
                            .withColumn(
                                      'queue_time',
                                      when(col('assigned') == 1, col('assign_time')).
                                      otherwise(col('abandon_time'))
                                     ) \
                            .withColumn(
                                      'cust_end_reason',
                                      when((col('endReason') == 205) | (col('endReason') == 210), col('endReasonDesc')).
                                      otherwise(lit(None))
                                      ) \
                            .withColumn(
                                      'cust_end_epoch',
                                      when((col('endReason') == 205) | (col('endReason') == 210), col('endtimeL')).
                                      otherwise(lit(None))
                                        ) \
                            .withColumn(
                                      'cust_end_dtm',
                                      when((col('endReason') == 205) | (col('endReason') == 210), col('endTime')).
                                      otherwise(lit(None))
                                      ) \
                            .withColumn(
                                      'end_reason',
                                      when((col('endReason') == 106) | (col('endReason') == 201), col('endReasonDesc')).
                                      otherwise(lit(None))
                                      ) \
                            .withColumn(
                                      'end_epoch',
                                      when((col('endReason') == 106) | (col('endReason') == 201), col('endTimeL')).
                                      otherwise(lit(None))
                                      ) \
                            .withColumn(
                                      'end_dtm',
                                      when((col('endReason') == 106) | (col('endReason') == 201), col('endTime')).
                                      otherwise(lit(None))
                                      ) \
                            .withColumn('agent_active_time', round(((col('endTimeL') - col('assign_epoch')) / 1000))) \
                            .withColumn('cust_active_time', round((col('endTimeL') - col('request_epoch')) / 1000)) \
                            .withColumn('transfer_agent_group', first(col('agent_group')).over(transfer_window)) \
                            .withColumn('transfer_agent_pein', first(col('agentLoginName')).over(transfer_window)) \
                            .withColumn('transfer_business_unit', first(col('business_unit')).over(transfer_window)) \
                            .withColumn('conversion', lit(None)) \
                            .withColumn('conversion_value', lit(None)) \
                            .withColumn('chat_started', lit(1))


In [130]:
history_chat = history_chat.select('engagementid',
                                   'leg_num',
                                   'business_unit',
                                   'agent_group',
                                   'business_rule',
                                   'launch_page_marker',
                                   'cust_lang',
                                   'cust_prov',
                                   'cust_company',
                                   'cust_email',
                                   'cust_phone',
                                   'request_epoch',
                                   'request_dtm',
                                   'assigned',
                                   'abandoned',
                                   'abandon_epoch',
                                   'abandon_dtm',
                                   'abandon_time',
                                   'assign_epoch',
                                   'assign_dtm',
                                   'assign_time',
                                   'assign_entertype',
                                   'queue_time',
                                   'cust_end_reason',
                                   'cust_end_epoch',
                                   'cust_end_dtm',
                                   'end_reason',
                                   'end_epoch',
                                   'end_dtm',
                                   'disposition_epoch',
                                   'disposition_dtm',
                                   'disposition_time',
                                   'handle_time',
                                   'transfer_agent_group',
                                   'transfer_agent_pein',
                                   'transfer_business_unit',
                                   'num_agent_freehand',
                                   'num_agent_scripted',
                                   'num_agent_responses',
                                   'num_cust_reponses',
                                   'initial_agent_response_time',
                                   'conversion',
                                   'conversion_value',
                                   'chat_started',
                                   'MAX_AGENT_RESPONSE_TIME',
                                   'TOTAL_RESPONSE_TIME',
                                   'AVG_RESPONSE_TIME',
                                   'NUM_CUST_TO_AGENT_RESPONSES')

In [131]:
history_chat.show()

+------------------+-------+--------------------+-----------------+-------------+--------------------+---------+---------+------------+----------+----------+-------------+--------------------+--------+---------+-------------+-----------+------------+-------------+--------------------+-----------+----------------+----------+--------------------+--------------+--------------------+----------+---------+-------+-----------------+---------------+----------------+-----------+--------------------+-------------------+----------------------+------------------+------------------+-------------------+-----------------+---------------------------+----------+----------------+------------+-----------------------+-------------------+-----------------+---------------------------+
|      engagementid|leg_num|       business_unit|      agent_group|business_rule|  launch_page_marker|cust_lang|cust_prov|cust_company|cust_email|cust_phone|request_epoch|         request_dtm|assigned|abandoned|abandon_epoch|a